In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [6]:
# Load the dataset

data  = pd.read_csv("Churn_Modelling.csv")
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [9]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [20]:
# One hot encode Geography column

one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [22]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [23]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [26]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns = one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.shape

(10000, 3)

In [27]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [28]:
## Combine the one hot encoded columns with the original data

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [29]:
## save pre-processings encoders 

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [30]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [32]:
## Divide the data into X and y

X= data.drop('Exited',axis=1)
y = data['Exited']

# Split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [33]:
print(X_train.shape)
print(X_test.shape)

(8000, 12)
(2000, 12)


In [35]:
# save the scaler
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Implementation

- Sequantial Network to Initialize
- Dense to create Hidden layers, say 64 for creating 64 neurons in a hidden layer
- Activation Function for each and every neuron
- Optimizers for back propagation --> updating the weights
- Loss Function 
- Metrics like accuracy, MSE, MAE, etc

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

import datetime

- Inputs are X_train.shape[1] that is for every obs we have 12 columns means 12 inputs
- Dense to create nodes

In [39]:
# Build our own ANN model

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)) ,#First Hidden Layer connected with Input Layer
    Dense(32,activation='relu'),   # Second Hidden Layer
    Dense(1,activation='sigmoid')   # Output Layer
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                832       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


- (12 x 64)+64+(64 x 32)+32+32+1  = 2945 are total number of parameters

In [43]:
import tensorflow

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [45]:
# Compile the model

model.compile(optimizer=opt,loss=loss,metrics=["accuracy"])
# or use
# model.compile(optimizer="adam",loss="binary_crossentropy",metric=["accuracy"])

In [52]:
## Setup the tensorboard

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)


- **Early Stopping:**   
A technique to monitor model performance and stop training when validation loss stops improving, preventing overfitting.

In [53]:
## Setup the Early stopping

early_stopping_callbacks = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [54]:
## Train the model

history = model.fit( X_train,y_train,validation_data=(X_test,y_test),epochs =100,
callbacks = [tensorflow_callback,early_stopping_callbacks])

Epoch 1/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3284 - accuracy: 0.8655 - val_loss: 0.3449 - val_accuracy: 0.8630
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3291 - accuracy: 0.8649 - val_loss: 0.3421 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3270 - accuracy: 0.8664 - val_loss: 0.3467 - val_accuracy: 0.8565
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3256 - accuracy: 0.8664 - val_loss: 0.3435 - val_accuracy: 0.8640
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3242 - accuracy: 0.8660 - val_loss: 0.3470 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3251 - accuracy: 0.8661 - val_loss: 0.3540 - val_accuracy: 0.8615
Epoch 7/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3213 - accuracy: 0.8689 - val_loss: 0.3545 - val_accuracy: 0.8585

In [55]:
# Save the model

model.save('model.h5')

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit20250102-143634/

# Predictions

In [78]:
from tensorflow.keras.models import load_model

In [ ]:
# Load the trained model,scalers and label encoder and one hot encoder

model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [87]:
# Example Data to test model/find predictions

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [93]:
# Onehot Encode 'Geography'

geo_encoded = one_hot_encoder_geo.transform([input_data['Geography']]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded,columns = one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Dell\python\NLP\ANN-classification\annvenv\ann_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [94]:
# combine one hot encoded columns with input data

input_df = pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [95]:
#Encode gender

input_df['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [96]:
input_df = input_df.drop(['Geography'],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [97]:
# Scaling data using scaler

input_scaled_df= scaler.transform(input_df)
input_scaled_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [99]:
# Predictions using model

prediction = model.predict(input_scaled_df)
prediction

1/1 [==============================] - 0s 73ms/step


array([[0.06200903]], dtype=float32)

In [101]:
prediction_proba = prediction[0][0]
prediction_proba

0.062009033

In [102]:
if prediction_proba > 0.5:
    print("Customer is likely to churn!")
else:
    print("Customer is not likely to churn.")

Customer is not likely to churn.


# End!